In [1]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_getr_d_paging_btm_next_10?ie=UTF8&reviewerType=all_reviews')

In [4]:
html_data = BeautifulSoup(driver.page_source, 'html.parser')

In [15]:
url = html_data.find('li',{'class' : 'a-last'}).a['href']
url

'/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2&reviewerType=all_reviews'

In [5]:
reviews = html_data.find_all('div',{'data-hook':'review'})

In [19]:
i = 0
names = []
ratings = []
rating_dates = []
titles = []
reviews_text = []
while True:
    i += 1
    url = 'https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=' + str(i)
    driver.get(url)
    time.sleep(5)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = html_data.find_all('div', {'data-hook': 'review'})
    for review in reviews:
        name = review.find('span', {'class': 'a-profile-name'}).text
        names.append(name.strip())
        rating = review.find('span', {'class': 'a-icon-alt'}).text
        ratings.append(rating)
        review_date = review.find('span', {'data-hook': 'review-date'}).text
        rating_dates.append(review_date)
        title = review.find('a', {'data-hook': 'review-title'}).text.strip()
        titles.append(title)
        review_text = review.find('span', {'data-hook': 'review-body'}).text.strip()
        reviews_text.append(review_text)
    next_page_element = html_data.find('li', {'class': 'a-last'}).find('a')
    if next_page_element:
        next_page_url = next_page_element['href']
        url = 'https://www.amazon.com' + next_page_url
    else:
        break
    print(url)


https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_3?ie=UTF8&pageNumber=3&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_4?ie=UTF8&pageNumber=4&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_5?ie=UTF8&pageNumber=5&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_6?ie=UTF8&pageNumber=6&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=cm_cr_arp_d_paging_btm_7?ie=UTF8&pageNumber=7&reviewerType=all_reviews
https://www.amazon.com/Apple-iPhone-15-128GB-Black/product-reviews/B0CMPMY9ZZ/ref=

In [21]:
data = pd.DataFrame({'profile_name': names,
                    'rating': ratings,
                     'review_date': rating_dates,
                    'title': titles,
                     'review_text': reviews_text
                    })

In [24]:
data.rating.value_counts()

rating
5.0 out of 5 stars    80
4.0 out of 5 stars    12
3.0 out of 5 stars     3
1.0 out of 5 stars     3
2.0 out of 5 stars     2
Name: count, dtype: int64

In [25]:
data.to_csv('iphone15_amazon_reviews')